In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
batch_size = 128
epochs = 10
seed = 1

In [7]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

In [9]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

In [16]:
X, y = iter(train_loader).next()
X.size(), y.size()

(torch.Size([128, 1, 28, 28]), torch.Size([128]))

In [65]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Unflatten(nn.Module):
    def __init__(self, channel, height, width):
        super(Unflatten, self).__init__()
        self.channel = channel
        self.height = height
        self.width = width

    def forward(self, input):
        return input.view(input.size(0), self.channel, self.height, self.width)

In [69]:
class VAE(nn.Module):
    
    def __init__(self):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(6272, 1024),
            nn.ReLU()
        )
        
        self.fc1 = nn.Linear(1024, 32)
        self.fc2 = nn.Linear(1024, 32)

        self.decoder = nn.Sequential(
            nn.Linear(32, 1024),
            nn.ReLU(),
            nn.Linear(1024, 6272),
            nn.ReLU(),
            Unflatten(128, 7, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc1(h), self.fc2(h)
        return mu, logvar
    
    def decode(self, z):
        z = self.decoder(z)
        return z

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [72]:
vae = VAE()
mu, logvar = vae.encode(X)
print(mu.size(), logvar.size())

z = vae.reparameterize(mu, logvar)
print(z.size())

Xhat = vae.decode(z)
print(Xhat.size())

torch.Size([128, 32]) torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128, 1, 28, 28])


In [73]:
h = nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1)(X)
print(h.size())
h = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)(h)
print(h.size())
h = h.view(h.size(0), -1)
print(h.size())
h = nn.Linear(6272, 1024)(h)
print(h.size())
h = nn.Linear(1024, 32)(h)
print(h.size())

torch.Size([128, 64, 14, 14])
torch.Size([128, 128, 7, 7])
torch.Size([128, 6272])
torch.Size([128, 1024])
torch.Size([128, 32])


In [74]:
h = nn.Linear(32, 1024)(h)
print(h.size())
h = nn.Linear(1024, 6272)(h)
print(h.size())
h = h.view(h.size(0), 128, 7, 7)
print(h.size())
h = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)(h)
print(h.size())
h = nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1)(h)
print(h.size())

torch.Size([128, 1024])
torch.Size([128, 6272])
torch.Size([128, 128, 7, 7])
torch.Size([128, 64, 14, 14])
torch.Size([128, 1, 28, 28])


In [79]:
vae = VAE()
Xhat, mu, logvar = vae(X)
print(X.size())
print(Xhat.size(), mu.size(), logvar.size())

torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28]) torch.Size([128, 32]) torch.Size([128, 32])


# Train

In [82]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [85]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [86]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item() / len(data)))
    
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [87]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], recon_batch.view(128, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), 'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')